In [1]:
import pandas as pd
import numpy as np
import os.path
import sqlite3

In [2]:
def select(query,path):
    
    conn = sqlite3.connect(path)
    cursor = conn.cursor()
    temp_df = pd.DataFrame(cursor.execute(query).fetchall())
    temp_df.columns = list(map(lambda x: x[0], cursor.description))
    conn.close()
    
    return temp_df.copy()

In [3]:
loans = select('SELECT * FROM LOAN_CLEAN',os.getcwd()+'/data/lending-club-loan-data/database2.sqlite')

In [4]:
# the data set has loans up through Dec. 2015, i.e. Jan. 2016 is "Now"

today = np.datetime64('2016-01-01')

In [5]:
# datetime arithmetic automatically counts the number of DAYS in between dates
# use approximate window of 30 days to count months

loans['mths_since_issue'] = (today - loans.issue_d.values.astype('datetime64[D]'))/np.timedelta64(30,'D')
loans['mths_since_issue'] = loans['mths_since_issue'].apply(lambda x: round(x))

In [6]:
# negative months left mean the loan has completed its term, i.e. no months left

def zero_out(x):
    if x < 0:
        return 0
    else:
        return x

loans['mths_left'] = loans['term'] - loans['mths_since_issue']
loans['mths_left'] = loans['mths_left'].apply(zero_out)

In [7]:
loans['%_term_completed'] = 1-(loans.mths_left/loans.term)

In [8]:
loans[['term','mths_since_issue','mths_left','%_term_completed','total_pymnt','funded_amnt']].head()

,term,mths_since_issue,mths_left,%_term_completed,total_pymnt,funded_amnt
0,36,50,0,1.000000,5861.071414,5000.0
1,60,50,10,0.833333,1008.710000,2500.0
2,36,50,0,1.000000,3003.653644,2400.0
3,36,50,0,1.000000,12226.302212,10000.0
4,60,50,10,0.833333,3242.170000,3000.0


In [9]:
#conn = sqlite3.connect(os.getcwd()+'\data\lending-club-loan-data\database2.sqlite')
#loans.to_sql('loan_final',conn,index=False)
#conn.close()